In [1]:
import json
import re

message = [ {"permission": "Deny", "rule": {"ruleID":"rule188","appName":"Its Too Hot","action_cmd":"sending_sms","action_device":"its-too-hot","trigger_device":"temp","trigger_attr":"temperature"}, "condition":None},{"permission": "Deny", "rule": {"ruleID":"rule186","appName":"Its Too Cold","action_cmd":"sending_sms","action_device":"its-too-cold","trigger_device":"temp","trigger_attr":"temperature"}, "condition":None},{"permission": "Allow", "rule": {"ruleID":"rule52","appName":"Darken Behind Me","action_cmd":"off","action_device":"smart-plug-1","trigger_device":"HueMotionSensor1","trigger_attr":"motion.inactive"}, "condition":None},{"permission": "Allow", "rule": {"ruleID":"rule189","appName":"Its Too Hot","action_cmd":"on","action_device":"fan","trigger_device":"temp","trigger_attr":"temperature >= 70"}, "condition":None},{"permission": "Allow", "rule": {"ruleID":"rule190","appName":"Its Too Hot","action_cmd":"sending_notification","action_device":"its-too-hot","trigger_device":"temp","trigger_attr":"temperature"}, "condition":None},{"permission": "Allow", "rule": {"ruleID":"rule187","appName":"Its Too Cold","action_cmd":"sending_notification","action_device":"its-too-cold","trigger_device":"temp","trigger_attr":"temperature"}, "condition":None},{"permission": "Allow", "rule": {"ruleID":"rule185","appName":"Its Too Cold","action_cmd":"on","action_device":"heater","trigger_device":"temp","trigger_attr":"temperature <= 70"}, "condition":None},{"permission": "Allow", "rule": {"ruleID":"rule0","appName":"OpenWindow","action_cmd":"on","action_device":"smart-plug-2","trigger_device":"smart-plug-1","trigger_attr":"switch.on"}, "condition":{"condition_device":"fan", "condition_cmd":"on", "condition_state":"is", "condition_time":None}},{"permission": "Allow", "rule": {"ruleID":"rule36","appName":"Brighten My Path","action_cmd":"on","action_device":"smart-plug-1","trigger_device":"HueMotionSensor1","trigger_attr":"motion.active"}, "condition":None}]

#Policies = json.dumps(message)
Policies = message
print(Policies)

[{'permission': 'Deny', 'rule': {'ruleID': 'rule188', 'appName': 'Its Too Hot', 'action_cmd': 'sending_sms', 'action_device': 'its-too-hot', 'trigger_device': 'temp', 'trigger_attr': 'temperature'}, 'condition': None}, {'permission': 'Deny', 'rule': {'ruleID': 'rule186', 'appName': 'Its Too Cold', 'action_cmd': 'sending_sms', 'action_device': 'its-too-cold', 'trigger_device': 'temp', 'trigger_attr': 'temperature'}, 'condition': None}, {'permission': 'Allow', 'rule': {'ruleID': 'rule52', 'appName': 'Darken Behind Me', 'action_cmd': 'off', 'action_device': 'smart-plug-1', 'trigger_device': 'HueMotionSensor1', 'trigger_attr': 'motion.inactive'}, 'condition': None}, {'permission': 'Allow', 'rule': {'ruleID': 'rule189', 'appName': 'Its Too Hot', 'action_cmd': 'on', 'action_device': 'fan', 'trigger_device': 'temp', 'trigger_attr': 'temperature >= 70'}, 'condition': None}, {'permission': 'Allow', 'rule': {'ruleID': 'rule190', 'appName': 'Its Too Hot', 'action_cmd': 'sending_notification', 'ac

In [2]:
def findPolicyNumber(policies):
    pattern = r'POLICY P'+ r'(\d+)'
    regex = re.compile(pattern,re.IGNORECASE)
    match = regex.findall(policies)
    if match:
        return int(match[-1])
    else:
        return 0

In [3]:
def PolicyGenerator(policyRules,policyFile):
    with open(policyFile,'r+') as policy:
        policies = policy.read()
        policyNumber = findPolicyNumber(policies)
        for rule in policyRules:
            policyNumber += 1
            newPolicy = '\n\nPOLICY P' + str(policyNumber) + ':\n'
            newPolicy += rule['permission'].upper() + '\t'
            triggerAction = rule['rule']
            condition = rule['condition']
            if condition is None:
                newPolicy += 'action_device = ' + triggerAction['action_device'] + ' AND\n'
                newPolicy += '\t\taction_command = ' + triggerAction['action_cmd'] + '\n'
                appName = triggerAction['appName'].replace(' ','-')
                newPolicy += 'ONLY IF automation_unit = ' + appName.lower()
            else:
                newPolicy += 'action_device = ' + triggerAction['action_device'] + ' AND\n'
                newPolicy += '\t\taction_command = ' + triggerAction['action_cmd'] + '\n'
                conditionObject = condition['condition_device'].replace(' ','-')
                
                # conditin event could be numerical such as "< 60"
                
                if condition["condition_state"] == "is":
                    state = 'state(' + conditionObject + ') = ' + condition['condition_cmd']
                else:
                    state = 'state(' + conditionObject + ') != ' + condition['condition_cmd']
                    
                if condition['condition_time'] is None or condition['condition_time'].lower() == '0':
                    newPolicy += 'ONLY IF ' + state
                else:
                    newPolicy += 'ONLY IF LASTLY(' + state + ') WITHIN [0,' + condition['condition_time'] + ']'
            
            
            policy.write(newPolicy)
        
            

policyFile = "mypolicy_userDefined.pat"
PolicyGenerator(Policies,policyFile)